In [ ]:
import numpy as np
import pandas as pd
import jieba
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
#load datasets
with open('chinese_stopwords.txt','r',encoding='utf-8') as file:
    stopwords=[i[:-1]for i in file.readlines()]
    
news=pd.read_csv('sqlResult.csv',encoding='gb18030')
print(news.shape)
print(news.head())
#preprocessing
print(news[news.content.isna()].head())
news=news.dropna(subset=['content'])
print(news.shape)

def split_text(text):
    text=text.replace('','')
    text=text.replace('\n','')
    text2=jieba.cut(text.strip())
    result=''.join([w for w in text2 if w not in stopwords])
    return result
print (news.iloc[0].content)
print(split_text(news.iloc[0].content))

import pickle,os
if not os.path.exists('corpus.pkl'):
    corpus=list(map(split_text,[str(i)for i in news.content]))
    print (corpus[0])
    print(len(corpus))
    with open('corpus.pkl','wb') as file:
        pickle.dump(corpus,file)
else:
    with open('corpus.pkl','rb') as file:
        corpus=pickle.load(file)

countvectorizer=CountVectorizer(encoding='gb18030', min_df=0.015)
tfidftransformer=TfidfTransformer()
countervector=countvectorizer.fit_transform(corpus)
tfidf=tfidftransformer.fit_transform(countervector)
print(tfidf.shape)

label=list(map(lambda source:1 if 'xinhua' in str(source) else 0,news.source))


In [ ]:
from sklearn.model.selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
x_train, x_test, y_train, y_test=train_test_split(tfidf).toarray(),label,test_size=0.3, random_state=33)
clf=MultinomialNB()
clf.fit(x_train,y_train )
prediction=clf.predict(tfidf.toarray())
labels=np.array(label)
compare_news_index=pd.DataFrame({'prediction':prediction,'labels':labels})
copy_news_index=compare_news_index[(compare_news_index['prediction']==1)&(compare_news_inedx['labels']==0)].index
xinhuashe_news_index=compare_news_index[(compare_news_index['labels']==1)].index


In [ ]:
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer
normalizer=Normalizer()
scaled_array=normalizer.fit_transform(tfidf.toarray())
if not os.path.exists('label.pkl'):
    kmeans=Kmeans(n_clusters=25)
    k_labels=kmeans.fit_predict(scaled_array)
    with open('label.pkl','wb') as file:
        pickle.dump(k_labels, file)
    print('k_labels.shape',k_labels.shape)
else:
    with open('label.pkl','rb') as file:
        k_labels=pickle.load(file)
    

In [ ]:
if not os.path.exists('id_class.pkl'): 
    id_class={index:class_ for index, class_ in enumerate(k_labels)}
    with open('id_class.pkl','wb') as file:
        pickle.dump(id_class, file)
else:
    with open('id_class.pkl','rb') as file:
        id_class=pickle.load(file)

In [ ]:
if not os.path.exists('class_id.pkl'):
    from collections import defaultdict
    class_id=defaultdict(set)
    for index, class_ in id_class.items():
        if index in xinhuashe_news_index.tolist():
            class_id[class_].add(index)
    with open('class_id.pkl','wb') as file:
        pickle.dump(class_id, file)
else:
    with open('class_id.pkl','rb') as file:
        id_class=pickle.load(file)

In [ ]:
def find_similar_text(cpindex,top=10): dist_dict={i:cosine_similarity(tfiddf[cpindex],tfidf[i]) for i in class_id[id_class[cpindex]]}
return sorted(dist_dict.items(),key=lambda x:x[1][0], reverse=True)[:top]
cpindex=3352 
similar_list=find_similar_text(cpindex) 
similar2=similar_list[0][0]
print('similar content\n',news.iloc[similar2].content)